## Search algorithms within Optuna

In this notebook, I will demo how to select the search algorithm with Optuna. We will compare the use of:

- Grid Search 
- Randomized search
- Tree-structured Parzen Estimators
- CMA-ES


We can select the search algorithm from the [optuna.study.create_study()](https://optuna.readthedocs.io/en/stable/reference/generated/optuna.study.create_study.html#optuna.study.create_study) class.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier

import optuna

In [4]:
# load dataset

breast_cancer_X, breast_cancer_y = load_breast_cancer(return_X_y=True)
X = pd.DataFrame(breast_cancer_X)
y = pd.Series(breast_cancer_y).map({0:1, 1:0})

X.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [5]:
# the target:
# percentage of benign (0) and malign tumors (1)

y.value_counts() / len(y)

0    0.627417
1    0.372583
dtype: float64

In [6]:
# split dataset into a train and test set

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

X_train.shape, X_test.shape

((398, 30), (171, 30))

## Define the objective function

This is the hyperparameter response space, the function we want to minimize.

In [7]:
# the objective function takes the hyperparameter space
# as input

def objective(trial):

    rf_n_estimators = trial.suggest_int("rf_n_estimators", 100, 1000)
    rf_criterion = trial.suggest_categorical("rf_criterion", ['gini', 'entropy'])
    rf_max_depth = trial.suggest_int("rf_max_depth", 1, 4)
    rf_min_samples_split = trial.suggest_float("rf_min_samples_split", 0.01, 1)
    
    model = RandomForestClassifier(
        n_estimators=rf_n_estimators,
        criterion=rf_criterion,
        max_depth=rf_max_depth,
        min_samples_split=rf_min_samples_split,
    )

    score = cross_val_score(model, X_train, y_train, cv=3)
    accuracy = score.mean()
    return accuracy

## Randomized Search

RandomSampler()

In [8]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.RandomSampler(),
)


study.optimize(objective, n_trials=5)

[I 2023-09-19 15:00:33,703] A new study created in memory with name: no-name-c1cd6854-419c-43c5-ac77-6287705f0d17
[I 2023-09-19 15:00:35,440] Trial 0 finished with value: 0.9171223513328776 and parameters: {'rf_n_estimators': 530, 'rf_criterion': 'gini', 'rf_max_depth': 1, 'rf_min_samples_split': 0.5495658093998813}. Best is trial 0 with value: 0.9171223513328776.
[I 2023-09-19 15:00:39,533] Trial 1 finished with value: 0.9347041847041847 and parameters: {'rf_n_estimators': 977, 'rf_criterion': 'entropy', 'rf_max_depth': 3, 'rf_min_samples_split': 0.08530102625612997}. Best is trial 1 with value: 0.9347041847041847.
[I 2023-09-19 15:00:40,151] Trial 2 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 212, 'rf_criterion': 'gini', 'rf_max_depth': 2, 'rf_min_samples_split': 0.979190748590727}. Best is trial 1 with value: 0.9347041847041847.
[I 2023-09-19 15:00:41,691] Trial 3 finished with value: 0.9171223513328776 and parameters: {'rf_n_estimators': 440, 'rf_cri

In [9]:
study.best_params

{'rf_n_estimators': 977,
 'rf_criterion': 'entropy',
 'rf_max_depth': 3,
 'rf_min_samples_split': 0.08530102625612997}

In [10]:
study.best_value

0.9347041847041847

In [11]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_rf_criterion,params_rf_max_depth,params_rf_min_samples_split,params_rf_n_estimators,state
0,0,0.917122,2023-09-19 15:00:33.705015,2023-09-19 15:00:35.439033,0 days 00:00:01.734018,gini,1,0.549566,530,COMPLETE
1,1,0.934704,2023-09-19 15:00:35.441064,2023-09-19 15:00:39.533783,0 days 00:00:04.092719,entropy,3,0.085301,977,COMPLETE
2,2,0.625636,2023-09-19 15:00:39.534782,2023-09-19 15:00:40.151091,0 days 00:00:00.616309,gini,2,0.979191,212,COMPLETE
3,3,0.917122,2023-09-19 15:00:40.151091,2023-09-19 15:00:41.691507,0 days 00:00:01.540416,entropy,1,0.364493,440,COMPLETE
4,4,0.625636,2023-09-19 15:00:41.692523,2023-09-19 15:00:43.139739,0 days 00:00:01.447216,entropy,1,0.696880,500,COMPLETE


## TPE

TPESampler is the default

In [12]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(),
)


study.optimize(objective, n_trials=5)

[I 2023-09-19 15:00:52,697] A new study created in memory with name: no-name-848b3e70-b941-4f1e-8817-3e2ce32734fc
[I 2023-09-19 15:00:55,429] Trial 0 finished with value: 0.9171413381939697 and parameters: {'rf_n_estimators': 840, 'rf_criterion': 'gini', 'rf_max_depth': 1, 'rf_min_samples_split': 0.3576700502122706}. Best is trial 0 with value: 0.9171413381939697.
[I 2023-09-19 15:00:56,208] Trial 1 finished with value: 0.937210450368345 and parameters: {'rf_n_estimators': 201, 'rf_criterion': 'entropy', 'rf_max_depth': 2, 'rf_min_samples_split': 0.15745568340980173}. Best is trial 1 with value: 0.937210450368345.
[I 2023-09-19 15:00:59,399] Trial 2 finished with value: 0.9371724766461608 and parameters: {'rf_n_estimators': 865, 'rf_criterion': 'gini', 'rf_max_depth': 3, 'rf_min_samples_split': 0.25808930498832133}. Best is trial 1 with value: 0.937210450368345.
[I 2023-09-19 15:01:02,695] Trial 3 finished with value: 0.9371724766461608 and parameters: {'rf_n_estimators': 837, 'rf_crit

In [13]:
study.best_params

{'rf_n_estimators': 201,
 'rf_criterion': 'entropy',
 'rf_max_depth': 2,
 'rf_min_samples_split': 0.15745568340980173}

In [14]:
study.best_value

0.937210450368345

## CMA-ES

CmaEsSampler

In [15]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.CmaEsSampler(),
)

study.optimize(objective, n_trials=5)

[I 2023-09-19 15:01:18,083] A new study created in memory with name: no-name-ea1b8752-ba4d-441a-8ff3-accb9dc8918b
[I 2023-09-19 15:01:19,015] Trial 0 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 310, 'rf_criterion': 'entropy', 'rf_max_depth': 2, 'rf_min_samples_split': 0.6557467903015166}. Best is trial 0 with value: 0.6256360598465861.
[W 2023-09-19 15:01:19,029] The parameter 'rf_criterion' in trial#1 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dynamic search space or `CategoricalDistribution`. You can suppress this warning by setting `warn_independent_sampling` to `False` in the constructor of `CmaEsSampler`, if this independent sampling is intended behavior.
[I 2023-09-19 15:01:21,572] Trial 1 finished with value: 0.9321409584567478 and parameters: {'rf_n_estimators': 687, 'rf_criterion': 'gini', 'rf_max_depth': 4, 'rf_min_samples_split': 0.3568

In [16]:
study.best_params

{'rf_n_estimators': 452,
 'rf_criterion': 'entropy',
 'rf_max_depth': 3,
 'rf_min_samples_split': 0.33755205148759904}

In [17]:
study.best_value

0.9396787423103211

## Grid Search

GridSampler()

We are probably not going to perform GridSearch with Optuna, but in case you wanted to, you need to add a variable with the space, with the exact values that you want to be tested.

In [18]:
search_space = {
    "rf_n_estimators": [100, 500, 1000],
    "rf_criterion": ['gini', 'entropy'],
    "rf_max_depth": [1, 2, 3],
    "rf_min_samples_split": [0.1, 1.0]
}


In [19]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.GridSampler(search_space),
)

study.optimize(objective)

[I 2023-09-19 15:01:44,659] A new study created in memory with name: no-name-1b99b545-19a2-4c1a-b456-d7936f838d7a
[I 2023-09-19 15:01:44,963] Trial 0 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 100, 'rf_criterion': 'entropy', 'rf_max_depth': 1, 'rf_min_samples_split': 1.0}. Best is trial 0 with value: 0.6256360598465861.
[I 2023-09-19 15:01:48,206] Trial 1 finished with value: 0.9171223513328776 and parameters: {'rf_n_estimators': 1000, 'rf_criterion': 'gini', 'rf_max_depth': 1, 'rf_min_samples_split': 0.1}. Best is trial 1 with value: 0.9171223513328776.
[I 2023-09-19 15:01:50,113] Trial 2 finished with value: 0.937210450368345 and parameters: {'rf_n_estimators': 500, 'rf_criterion': 'entropy', 'rf_max_depth': 2, 'rf_min_samples_split': 0.1}. Best is trial 2 with value: 0.937210450368345.
[I 2023-09-19 15:01:51,797] Trial 3 finished with value: 0.9146160856687172 and parameters: {'rf_n_estimators': 500, 'rf_criterion': 'entropy', 'rf_max_depth': 1, 'rf_

[I 2023-09-19 15:02:48,253] Trial 34 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 1000, 'rf_criterion': 'gini', 'rf_max_depth': 2, 'rf_min_samples_split': 1.0}. Best is trial 10 with value: 0.942241968557758.
[I 2023-09-19 15:02:51,235] Trial 35 finished with value: 0.6256360598465861 and parameters: {'rf_n_estimators': 1000, 'rf_criterion': 'entropy', 'rf_max_depth': 3, 'rf_min_samples_split': 1.0}. Best is trial 10 with value: 0.942241968557758.


In [22]:
study.best_params

{'rf_n_estimators': 100,
 'rf_criterion': 'gini',
 'rf_max_depth': 3,
 'rf_min_samples_split': 0.1}

In [23]:
study.best_value

0.942241968557758